# Part 1 Prepare csv File

In [8]:
import pandas as pd
import os
import csv

In [6]:
all_entries = []
for file in os.listdir(r'C:/Users/lenovo/Downloads/udacity_data_engineering-master/udacity_data_engineering-master/p2_data_modeling_cassandra/event_data'):
    with open(os.path.join(r'C:/Users/lenovo/Downloads/udacity_data_engineering-master/udacity_data_engineering-master/p2_data_modeling_cassandra/event_data',file),encoding='utf-8') as f:
        reader = csv.reader(f)
        next(reader)
        for row in reader:
            all_entries.append(row)
len(all_entries)

8056

In [11]:
with open(os.path.join(os.getcwd(),'event_datafile_new.csv'),'w',encoding='utf-8',newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for entry in all_entries:
        if entry[0].strip()!='':
            writer.writerow((entry[0],entry[2],entry[3],entry[4],entry[5],entry[6],entry[7],entry[8],
                             entry[12],entry[13],entry[16]))

#### The new file `event_datafile_new.csv` is located within the Workspace directory. The file contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

# Part 2 Define Cassandra Database

In [1]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

### Creating Database

In [15]:
query = '''
Create Keyspace if not exists sparkifydb
with replication = {'class':'SimpleStrategy','replication_factor':1}
'''
try:
    session.execute(query)
except Exception as e:
    print(e)

### Setting Keyspace

In [6]:
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

### Query 1 Creating table using session_id and item_in_session

In [18]:
query = '''
Create table if not exists song_in_session
(session_id INT, item_in_session INT, artist_name VARCHAR, song_title VARCHAR, song_duration DECIMAL,
PRIMARY KEY (session_id, item_in_session))
'''
try:
    session.execute(query)
except Exception as e:
    print(e)

### Populating the first table using csv

In [22]:
with open('event_datafile_new.csv',encoding='utf-8',newline='') as f:
    reader = csv.reader(f)
    next(reader)
    query = '''
    Insert into song_in_session (session_id, item_in_session, artist_name, song_title, song_duration) 
    Values (%s,%s,%s,%s,%s)'''
    for entry in reader:
        try:
            session.execute(query,(int(entry[8]), int(entry[3]), entry[0], entry[9], float(entry[5])))
        except Exception as e:
            print(e)
            break

In [25]:
row = session.execute('Select * from song_in_session where session_id = 338 and item_in_session = 4')
testing_df = pd.DataFrame(list(row))
testing_df.columns = ['Session_id','item_in_session','artist_name','song_title','song_length']

In [26]:
testing_df

,Session_id,item_in_session,artist_name,song_title,song_length
0,338,4,Faithless,495.3073,Music Matters (Mark Knight Dub)


### Query 2
**Requested information:** 
Give only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

**Approach:**
Since we're seeking results for a specific user and session, we'll use both `user_id` and `session_id` as a [composite partition key](https://docs.datastax.com/en/archived/cql/3.3/cql/cql_using/useCompositePartitionKeyConcept.html). And, because we need to sort the results by `item_in_session`, we'll use that as the clustering column. From the query results, we'll only display `artist_name`, `song_title`, `user_first_name` and `user_last_name` within a dataframe. 

In [7]:
query = '''
Create table if not exists song_in_user_session
(user_id int, session_id int, artist varchar, song varchar, first_name varchar, last_name varchar,
primary key((user_id,session_id),song))
'''
try:
    session.execute(query)
except Exception as e:
    print(e)

### Putting the data in the song_in_user_session table from csv

In [10]:
query = '''
Insert into song_in_user_session 
(user_id, session_id, artist, song, first_name, last_name) 
values (%s,%s,%s,%s,%s,%s)
'''
with open('event_datafile_new.csv',encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        try:
            session.execute(query,(int(row[10]),int(row[8]),row[0],row[9],row[1],row[4]))
        except Exception as e:
            print(e)
            break

### Creating Dataframe from the table

In [56]:
rows = session.execute('Select * from song_in_user_session where user_id = 10 and session_id = 182')
df = pd.DataFrame(list(rows))
df.style.hide_index()

user_id,session_id,song,artist,first_name,last_name
10,182,Catch You Baby (Steve Pitron & Max Sanna Radio Edit),Lonnie Gordon,Sylvie,Cruz
10,182,Greece 2000,Three Drives,Sylvie,Cruz
10,182,Keep On Keepin' On,Down To The Bone,Sylvie,Cruz
10,182,Kilometer,Sebastien Tellier,Sylvie,Cruz


### Query 3
**Requested information:** 
Give every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

**Approach:**
Since we want the results to be unique based on a specific song, we'll use `song_title` as the partition key. And since we're looking for the set of users who listened to that song, we'll use `user_id` as the clustering column. From the query results, we'll only display `user_first_name` and `user_last_name` within a dataframe. 

In [59]:
query = '''CREATE TABLE IF NOT EXISTS user_song_listen 
(user_id INT, user_first_name VARCHAR, user_last_name VARCHAR, song_title VARCHAR, 
PRIMARY KEY (song_title, user_id))'''
try:
    session.execute(query)
except Exception as e:
    print(e)

In [62]:
query = '''
Insert into user_song_listen 
(user_id, user_first_name, user_last_name, song_title) 
values (%s,%s,%s,%s)
'''
with open('event_datafile_new.csv',encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        try:
            session.execute(query,(int(row[10]),row[1],row[4],row[9]))
        except Exception as e:
            print(e)
            break

In [64]:
records = session.execute('Select * from user_song_listen where song_title = \'All Hands Against His Own\'')
df = pd.DataFrame(list(records))
df

,song_title,user_id,user_first_name,user_last_name
0,All Hands Against His Own,29,Jacqueline,Lynch
1,All Hands Against His Own,80,Tegan,Levine
2,All Hands Against His Own,95,Sara,Johnson
